In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.preprocessing import StandardScaler as Scaler
from sklearn.ensemble import RandomForestRegressor
# from skrebate import ReliefF
from ReliefF import ReliefF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import joblib

"""

코드 실행 방법
1. 밑의 joblib.load 부분을 찾아, 코드 파일과 함께 압축되어 있던 .pkl 파일의 위치를 입력해준다.
2. 실행한다. 라이브러리가 설치되어 있지 않으면 pip install을 통해 설치한다.

"""

"""

데이터 전처리 부분

"""

encoder = LE()
# 파일 읽어오기
df = pd.read_csv("/Users/yddook/ukgrade/prog/jobgot/머러기 데이터셋 - 시트1 (2).csv")

# 행 이름 다시 설정하기
new_col = ['지역', '축제명', '축제유형', '축제사무국상설화여부', '사업위탁운영여부', '2017b', '2018b', '2019b', '2021b', '2022b', '2023b', '2017v', '2018v', '2019v', '2021v', '2022v'] # '20__b'는 20__년 예산, '20__v'는 20__년 방문객임
df.columns = new_col
df.drop([0], axis=0, inplace=True)

# 쓸모 없는 0번째 행을 제거하면서 인덱스가 1부터 시작하게 되기 때문에, 0부터 시작하도록 인덱스 초기화
df.index = [i for i in range(100)]

# 원활한 데이터 가공을 위해 dataframe의 각 열의 자료형 변경
df = df.astype({'2018b':'float64'}) # 기존의 정수에서 소수로 변경
num_col = ['2017b', '2021b', '2022b', '2017v', '2018v', '2019v']
df[num_col] = df[num_col].apply(pd.to_numeric, errors='coerce') # 예산과 방문객은 숫자여야 하기 때문에 기존의 object에서 소수로 자료형을 변경, 이 과정에서 '(미개최)'같은 문자열이나 빈칸이 제거됨

# 데이터 정규화와 표준화 -> 값의 크기에 의한 영향 줄이기
# 표준화는 예산에 대해, 방문자 수에 대해 각각 적용해야 함

# 표준화 함수
def standardize_all_elements(df):
    scaler = Scaler()
    df_flattened = df.values.flatten().reshape(-1, 1)
    scaler.fit(df_flattened)
    mean, std = scaler.mean_, scaler.scale_
    df_standardized = scaler.transform(df_flattened)
    df_standardized = df_standardized.reshape(df.shape)
    df_standardized = pd.DataFrame(df_standardized, columns=df.columns)
    return df_standardized, mean, std

# dataframe에서 0의 값을 없애는 과정
numeric_columns = df.select_dtypes(include='number')
df_number = numeric_columns.replace(0.0, np.nan)

# 예산과 방문자를 표준화하고, 방문자 표준화에서는 모델이 예상한 값을 방문자 수치로 환산하기 위한 평균과 표준편차를 구함
df_1 = standardize_all_elements(df_number[['2017b', '2018b', '2019b', '2021b', '2022b', '2023b']])[0]
df_2, mean, std = standardize_all_elements(df_number[['2017v', '2018v', '2019v', '2021v', '2022v']])

df_number = pd.concat([df_1, df_2], axis=1)

df = df.drop(['2017b', '2018b', '2019b', '2021b', '2022b', '2023b', '2017v', '2018v', '2019v', '2021v', '2022v'], axis=1)
df = pd.concat([df, df_number], axis=1)
#df.to_csv("/Users/yddook/ukgrade/prog/jobgot/test433.csv")

# 인덱스를 축제명으로 변경
df.set_index(keys=['축제명'],inplace=True)

df = df.astype({'축제사무국상설화여부':'int64'}) # 기존의 문자에서 정수로 변경

df = df.fillna(0) # 위에서 예산과 방문객을 소수형을 바꾸면서 숫자가 아닌 칸들은 NaN(Not a Number)으로 내용이 바뀌는데, 이 칸들을 모두 0으로 채움

# 지역과 축제유형을 문자열에서 숫자로 변형
le_loc = LE()
new_loc = le_loc.fit_transform(df['지역'])
le_type = LE()
new_type = le_loc.fit_transform(df['축제유형'])
df['지역'] = new_loc
df['축제유형'] = new_type

# 범주형 변수를 one-hot vector를 통해 수치형 변수로 변형 (그래야 모델이 범주형 변수를 학습 가능)
df = pd.get_dummies(df, columns=['지역', '축제유형', '사업위탁운영여부'])

'''
지역 순서 (0, 1, 2, ...)
['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']

축제유형 순서 (0, 1, 2, ...)
['기타', '문화예술', '생태자연', '전통역사', '특산물']

사업위탁운영여부 순서 (0, 1, 2)
[전체위탁, 부분위탁, 자체추진]
'''

#get_dummies로 범주형 변수를 one-hot representation으로 변형하며 바뀐 열 순서를 원하는 순서로 변형
new_col_again = ['지역_0', '지역_1', '지역_2', '지역_3', '지역_4', '지역_5', '지역_6', '지역_7', '지역_8', '지역_9', '지역_10', '지역_11', '지역_12', '지역_13', '지역_14', '지역_15', '지역_16', '축제유형_0', '축제유형_1', '축제유형_2', '축제유형_3', '축제유형_4', '축제사무국상설화여부', '사업위탁운영여부_0', '사업위탁운영여부_1', '사업위탁운영여부_2', '2017b', '2018b', '2019b', '2021b','2022b', '2023b', '2017v', '2018v', '2019v', '2021v', '2022v']
df = df.reindex(columns=new_col_again)

#학습을 위한 데이터셋 만들기 / data: 독립변인, target: 종속변인
data = []
target = []

#df.to_csv("/Users/yddook/ukgrade/prog/jobgot/test51.csv")

column_names = ['지역_0', '지역_1', '지역_2', '지역_3', '지역_4', '지역_5', '지역_6', '지역_7', '지역_8', '지역_9', '지역_10', '지역_11', '지역_12', '지역_13', '지역_14', '지역_15', '지역_16', '축제유형_0', '축제유형_1', '축제유형_2', '축제유형_3', '축제유형_4', '축제사무국상설화여부', '사업위탁운영여부_0', '사업위탁운영여부_1', '사업위탁운영여부_2','올해예산', '작년방문객', '올해방문객']
df_fff = pd.DataFrame(columns=column_names)

for index, row in df.iterrows():
    for i in range(4):
        #전년도 방문객을 독립변인으로 하기에 2018, 2019, 2021, 2022, 총 4개에 대한 예측이 가능함
        candidate1 = row[27+i]-0 #올해 예산 -> 0인 경우 사례가 될 수 없음
        candidate2 = row[32+i]-0 #작년 방문객 -> 0인 경우 사례가 될 수 없음
        candidate3 = row[33+i]-0 #올해 방문객 -> 0인 경우 사례가 될 수 없음
        if candidate1 != 0 and (candidate2 != 0 and candidate3 != 0):
            target.append(candidate3)
            row_data = row[0:26].values.tolist()
            row_data.append(row[27+i])
            row_data.append(row[32+i])
            data.append(row_data)

            # # RReliefF를 위한 데이터 처리 -> 취소
            # row_data.append(candidate3)
            # new_row = pd.DataFrame([row_data], columns=df_fff.columns)
            # df_fff = pd.concat([df_fff, new_row], axis=0, ignore_index=True)

# # code for debugging
# print((data))
# print(len(data)) #data가 적절히 채워졌는지 확인
# print(len(target)) #target가 적절히 채워졌는지 확인

"""

데이터를 훈련 세트와 검증 세트로 나눔
머신러닝 모델을 학습시킴

"""

# 훈련 세트와 검증 세트로 데이터 나누기 -> 80%의 데이터를 학습에 이용, 나머지 20%는 검증에 이용
data_train, data_test, target_train, target_test=train_test_split(data, target, test_size=0.2)

# # 랜덤 포레스트 모델을 학습시키기
# reg=RandomForestRegressor()
# reg.fit(data_train, target_train)

# 모델 불러오기 
reg = joblib.load('/Users/yddook/downloads/final_model.pkl')

# 학습시킨 모델을 바탕으로 test set에 대해 예측하기
y_pred=reg.predict(data_test)
mse = mean_squared_error(target_test, y_pred)

# mean과 standard deviation을 통해 예측치를 예상 관광객 수로 변형하기
# mean = df[]
print("Mean Squared Error:", mse)
print("Mean, STD:", mean, std)

# 역변환 함수
def inverse_transform(standardized_value, std, mean):
    original_value = standardized_value * std + mean
    return round(float(original_value), 3)

org_y = []
for pred_value in y_pred.tolist():
    org_value = inverse_transform(pred_value, std, mean)
    org_y.append(org_value)

org_target = []
for test_value in target_test:
    org_value = inverse_transform(test_value, std, mean)
    org_target.append(org_value)

print(org_y, "\n\n")
print(org_target)

importances = reg.feature_importances_

# 중요도 순서대로 정렬된 인덱스
indices = importances.argsort()[::-1]

# 변수 중요도와 순위 출력
for i, idx in enumerate(indices):
    feature = df_fff.columns[idx]
    importance = importances[idx]
    rank = i + 1
    print(f"Rank {rank}: {feature} (Importance: {importance})")

# df.to_csv("/Users/yddook/ukgrade/prog/jobgot/test4.csv") 

def r_squared(y_true, y_pred):
    """결정계수 계산 함수 -> 모델이 얼마나 적절한지를 표현 / 음수면 모델이 데이터의 평균보다 못한 예측을 하는 경우"""
    y_true = np.array(y_true)  # 리스트를 numpy 배열로 변환
    y_pred = np.array(y_pred)  # 리스트를 numpy 배열로 변환
    y_mean = np.mean(y_true)  # 실제 종속 변수의 평균
    ss_total = np.sum((y_true - y_mean)**2)  # 총 제곱합
    ss_residual = np.sum((y_true - y_pred)**2)  # 잔차 제곱합
    r2 = 1 - (ss_residual / ss_total)  # 결정계수 계산
    return r2

r2_score = r_squared(org_target, org_y)
print("R-squared:", r2_score)

# 시각화
plt.scatter(org_target, org_y)
plt.plot(org_target, org_target, color='r', linestyle='--')
plt.xlabel("True Values") # 참값
plt.ylabel("Predicted Values") # 모델의 예측치
plt.title("True Values vs Predicted Values")
plt.show() # 점들이 붉은 직선 (y=x)에 모여 있을수록 예측치와 참값이 비슷하다는 의미임.

""" 중요 -> 저장 여부 구분 필요 """
# # 모델을 저장, 저장할 때에만 주석 해제
# joblib.dump(reg, '/Users/yddook/final_model.pkl')















""" ReliefF 수행을 위해 코드를 짰지만 실패.
# ReliefF 수행 -> 변수의 중요도 분석
rf = ReliefF()
rf.fit_transform(data_train, data_test)

df['combined_categorical_1'] = df['지역_0'].astype(str) + df['지역_1'].astype(str) + df['지역_2'].astype(str) + df['지역_3'].astype(str) + df['지역_4'].astype(str) + df['지역_5'].astype(str) + df['지역_6'].astype(str) + df['지역_7'].astype(str) + df['지역_8'].astype(str) + df['지역_9'].astype(str) + df['지역_10'].astype(str) + df['지역_11'].astype(str) + df['지역_12'].astype(str) + df['지역_13'].astype(str) + df['지역_14'].astype(str) + df['지역_15'].astype(str) + df['지역_16'].astype(str)

df['combined_categorical_2'] = df['축제유형_0'].astype(str) + df['축제유형_1'].astype(str) + df['축제유형_2'].astype(str) + df['축제유형_3'].astype(str) + df['축제유형_4'].astype(str)

rf_data_X = df.drop(['target_variable'], axis=1)  # 특성 변수들

# RReliefF를 위한 데이터셋 준비
X = df_fff.drop('올해방문객', axis=1)  # 특성 변수들
print(X)
X.to_csv("/Users/yddook/ukgrade/prog/jobgot/test4444.csv") 
y = df_fff['올해방문객']  # 목표 변수
print(y)

# RReliefF 객체 생성
relieff = ReliefF(n_neighbors=5)

# 특성 선택 수행
X_selected = relieff.fit_transform(X, y)

# 선택된 특성 인덱스 가져오기
selected_feature_indices = relieff.top_features_

# 선택된 특성들 출력
selected_features = X.columns[selected_feature_indices]
print("Selected Features:", selected_features)
"""